In [15]:
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt
import os
import math
import functionsMaths as f

In [66]:
#Inicializamos los pesos de la Capa 1
w = np.random.random(500)
w11 = w[:100]
w12 = w[100:200]
w13 = w[200:300]
w14 = w[300:400]
w15 = w[400:500]
W1=[w11,w12,w13,w14,w15]

#Inicializamos los pesos de la Capa 2
w = np.random.random(25)
w21 = w[:5]
w22 = w[5:10]
w23 = w[10:15]
w24 = w[15:20]
w25 = w[20:25]
W2 = [w21,w22,w23,w24,w25] 

#Incializamos los pesos de la Capa 3
w = np.random.random(15)
w31 = w[:5]
w32 = w[5:10]
w33 = w[10:15]
W3 = [w31,w32,w33]

#Bias
b = np.random.random(13)

b1= b[:5]
b2 = b[5:10]
b3 = b [10:]


# pesos
w1 = np.array(W1)
w2 = np.array(W2)
w3 = np.array(W3)







In [73]:
data = (pd.read_csv(os.path.join(os.path.abspath(''),"data","distorsionadas",'500','letras.csv'),sep=';',header=None)).to_numpy()
#Convertimos el dataframe en array numpy
data_train = data[:int(len(data)*0.8)]
data_test = data[int(len(data)*0.8)+1:int(len(data)*0.8)+int(len(data)*0.15)]
data_validation = data[int(len(data)*0.8)+int(len(data)*0.15)+1:99]
len(data_test)



74

In [55]:
def divX_Y(letras):
    for letra in letras:
        x_train = letra[:100]
        y_train = letra[100:]
    return x_train,y_train

In [74]:
#Dividiendo data train en una tupla (entrada,clase)
letras_train = []
data_test_predict = []
for letra in data_train:
    x_train = letra[:100]
    y_train = letra[100:]
    letras_train.append((x_train, y_train))

for letra in data_test:
    x_train = letra[:100]
    y_train = letra[100:]
    data_test_predict.append((x_train, y_train))

len(data_test_predict)

74

In [325]:
err = []

In [75]:
for e in range(50):
    np.random.shuffle(letras_train)
    for i in range(len(letras_train)):
                # feedforward
                #Primer capa
                ys1 =[]
                z1 = []
                deltas = []
                zs = []
                activaciones = [letras_train[i][0]]
                for j in range(len(w1)):
                    #Suma ponderada
                    Z1 = np.dot(letras_train[i][0],w1[j])
                    #Funcion de activacion
                    ys1.append(f.lineal(Z1+b1[j]))
                    z1.append(Z1)
                
                z1 = np.array(z1)
                ys1 = np.array(ys1)

                activaciones.append(ys1)
                zs.append(z1)
                #Segunda capa
                ys2 = []
                z2 = []
                for j in range(len(w2)):
                    #Suma ponderada
                    Z2 = np.dot(ys1,w2[j])
                    #Funcion de activacion
                    ys2.append(f.lineal(Z2+b2[j]))
                    z2.append(Z2)
                
                z2 = np.array(z2)
                ys2 = np.array(ys2)
                
                activaciones.append(ys2)
                zs.append(z2)
                #Capa de salida
                ys3 = []
                z3 = []
                for j in range(len(w3)):
                    #Suma ponderada
                    Z3 = np.dot(ys2,w3[j])
                    #Funcion de activacion
                    ys3.append(f.sigmoide(Z3+b3[j]))
                    z3.append(Z3)
                #print("Capa salida: ", ys3)
                #!!-----Calculo del error-----!!!

                ys3 = np.array(ys3)
                z3 = np.array(z3)

                activaciones.append(ys3)
                zs.append(z3)

                Ye = np.array(letras_train[i][1])
                #----------------------------------------------------#

                delta = f.cost_derivate(activaciones[3], Ye)*f.sigmoide_derivate(z3)
                delta = np.array(delta)
                b3 -= delta*0.3 

                deltas.append(delta)
                ##print(ys3, letras_train[i][1],delta)
                #CAPA OCULTA N° 2
                delta2 = np.dot(np.transpose(w3),deltas[0])*f.lineal_derivate()
                b2 -= delta2*0.3

                deltas.append(delta2)

                #CAPA OCULTA N° 1
                delta1 = np.dot(np.transpose(w2),deltas[1])*f.lineal_derivate()
                b1 -= delta1*0.3

                deltas.append(delta1)

                #Backforward
                for j in range(len(w3)):
                    for k in range(len(w2)):
                        w3[j][k] -= activaciones[2][k]*deltas[0][j]*0.3
                
                for j in range(len(w2)):
                    for k in range(len(w1)):
                        w2[j][k] -= activaciones[1][k]*deltas[1][j]*0.3

                for j in range(len(w1)):
                    for k in range(100):
                        w1[j][k] -= activaciones[0][k]*deltas[2][j]*0.3 
               

In [338]:
def backforward(W,activations, deltas,num_layers, lr, m):
    i = 0
    for w in W:
        x,y = w.shape
        for j in range(x):
            for k in range(k):
                w[j][k] -= activations[i][k]*deltas[i][j]*lr*m
                




0

In [77]:

## PREDICT 
letra = ["b","d","f"]
porcentaje = 0
for i in range(len(data_test_predict)):
    ys1 =[]
    z1 = []
    for j in range(len(w1)):
                #Suma ponderada
        Z1 = np.dot(data_test_predict[i][0],w1[j])
                #Funcion de activacion
        ys1.append(f.lineal(Z1+b1[j]))
        z1.append(Z1)
                
    z1 = np.array(z1)
    ys1 = np.array(ys1)

    #print("Primer capa: ", ys1)
    #Segunda capa
    ys2 = []
    z2 = []
    for j in range(len(w2)):
        #Suma ponderada
        Z2 = np.dot(ys1,w2[j])
        #Funcion de activacion
        ys2.append(f.lineal(Z2+b2[j]))
        z2.append(Z2)
                
    z2 = np.array(z2)
    ys2 = np.array(ys2)

    #print("2da capa: ", ys2)
    #Capa de salida
    ys3 = []
    z3 = []
    for j in range(len(w3)):
        #Suma ponderada
        Z3 = np.dot(ys2,w3[j])
        #Funcion de activacion
        ys3.append(f.sigmoide(Z3+b3[j]))
        z3.append(Z3)
                #print("Capa salida: ", ys3)
                #!!-----Calculo del error-----!!!

    ys3 = np.array(ys3)
    z3 = np.array(z3)

    if(np.argmax(ys3) == np.argmax(data_test_predict[i][1])):
        porcentaje += 1
    print("Prediccion: {} letra: {} ---- valor real: {}, letra: {}".format( np.argmax(ys3)+1,letra[ np.argmax(ys3)],np.argmax(data_test_predict[i][1])+1,letra[np.argmax(data_test_predict[i][1])] ))
print("Porcentaje predicho: {}%".format((porcentaje/len(data_test_predict))*100))

[[ 3.5088588   0.91004918 -1.85936725  4.27909434 -3.34648186]
 [ 0.15455739 -0.51043098  2.19591025 -1.33406801  6.32827946]
 [-3.05627075 -1.9908203  -0.27726888 -3.74062562 -3.42119901]]
Prediccion: 3 letra: f ---- valor real: 3, letra: f
Prediccion: 1 letra: b ---- valor real: 1, letra: b
Prediccion: 2 letra: d ---- valor real: 2, letra: d
Prediccion: 3 letra: f ---- valor real: 3, letra: f
Prediccion: 2 letra: d ---- valor real: 2, letra: d
Prediccion: 2 letra: d ---- valor real: 2, letra: d
Prediccion: 1 letra: b ---- valor real: 1, letra: b
Prediccion: 1 letra: b ---- valor real: 1, letra: b
Prediccion: 2 letra: d ---- valor real: 2, letra: d
Prediccion: 1 letra: b ---- valor real: 1, letra: b
Prediccion: 3 letra: f ---- valor real: 3, letra: f
Prediccion: 1 letra: b ---- valor real: 1, letra: b
Prediccion: 2 letra: d ---- valor real: 2, letra: d
Prediccion: 1 letra: b ---- valor real: 1, letra: b
Prediccion: 1 letra: b ---- valor real: 1, letra: b
Prediccion: 1 letra: b ---- va